# Swarm Defense Game - Configuration Demo

This notebook demonstrates how to create games with different configurations.


In [1]:
# In a Jupyter notebook, to automatically reload libraries when their source code changes,
# use the IPython autoreload extension. This helps you see changes in swarm_defense_game without restarting the kernel.

%load_ext autoreload
%autoreload 2

import sys
import pyspiel

# Add the game directory to path to ensure swarm_defense_game is discoverable
sys.path.insert(0, '.')
import swarm_defense_game



## 1. Small Game (Fast Training/Testing)

Small grid, fewer units - good for quick experiments and debugging.


In [2]:
# Create a small game
small_game = pyspiel.load_game("swarm_defense", {
    "grid_rows": 12,
    "grid_cols": 12,
    "num_targets": 2,
    "num_ad_units": 1,
    "num_attacking_drones": 5,
    "num_interceptors": 3,
})

# Display configuration
config = small_game.config
print("=== SMALL GAME CONFIG ===")
print(f"Grid Size: {config.grid_rows}x{config.grid_cols}")
print(f"Targets: {config.num_targets}")
print(f"AD Units: {config.num_ad_units}")
print(f"Attacking Drones: {config.num_attacking_drones}")
print(f"Interceptors: {config.num_interceptors}")
print(f"\nAction Space Size: {small_game.action_info['num_distinct_actions']}")
print(f"Max Game Length: {config.compute_max_game_length()}")


=== SMALL GAME CONFIG ===
Grid Size: 12x12
Targets: 2
AD Units: 1
Attacking Drones: 5
Interceptors: 3

Action Space Size: 208
Max Game Length: 13


In [3]:
# Show the state space vector for the small_game
state = small_game.new_initial_state()

# OpenSpiel error "player >= 0" often means the state is not at a player turn (chance or terminal).
# observation_tensor() should be called as a player, not in a chance/terminal state.

if not state.is_terminal() and not state.is_chance_node():
    vector = state.observation_tensor()
    print("\n=== SMALL GAME State Space Vector ===")
    print(vector)
    print(f"State vector length: {len(vector)}")
else:
    print("State is not at a regular player turn. Skipping observation_tensor().")


State is not at a regular player turn. Skipping observation_tensor().


## 2. Large Game (Realistic Scenario)

Larger grid, more units - closer to realistic military scenarios.


In [4]:
# Create a large game
large_game = pyspiel.load_game("swarm_defense", {
    "grid_rows": 32,
    "grid_cols": 24,
    "num_targets": 5,
    "num_ad_units": 4,
    "num_attacking_drones": 20,
    "num_interceptors": 12,
    "ad_kill_rate": 3.0,
    "interceptor_reward": 1.5
})

# Display configuration
config = large_game.config
print("=== LARGE GAME CONFIG ===")
print(f"Grid Size: {config.grid_rows}x{config.grid_cols}")
print(f"Targets: {config.num_targets}")
print(f"AD Units: {config.num_ad_units}")
print(f"Attacking Drones: {config.num_attacking_drones}")
print(f"Interceptors: {config.num_interceptors}")
print(f"AD Kill Rate: {config.ad_kill_rate}")
print(f"Interceptor Reward: {config.interceptor_reward}")
print(f"\nAction Space Size: {large_game.action_info['num_distinct_actions']}")
print(f"Max Game Length: {config.compute_max_game_length()}")


=== LARGE GAME CONFIG ===
Grid Size: 32x24
Targets: 5
AD Units: 4
Attacking Drones: 20
Interceptors: 12
AD Kill Rate: 3.0
Interceptor Reward: 1.5

Action Space Size: 1271
Max Game Length: 46


## 3. Comparison - Side by Side


In [5]:
import pandas as pd

# Create comparison dataframe
comparison = pd.DataFrame({
    'Parameter': [
        'Grid Rows',
        'Grid Cols',
        'Targets',
        'AD Units',
        'Drones',
        'Interceptors',
        'Action Space',
        'Max Game Length'
    ],
    'Small': [
        small_game.config.grid_rows,
        small_game.config.grid_cols,
        small_game.config.num_targets,
        small_game.config.num_ad_units,
        small_game.config.num_attacking_drones,
        small_game.config.num_interceptors,
        small_game.action_info['num_distinct_actions'],
        small_game.config.compute_max_game_length()
    ],
    'Large': [
        large_game.config.grid_rows,
        large_game.config.grid_cols,
        large_game.config.num_targets,
        large_game.config.num_ad_units,
        large_game.config.num_attacking_drones,
        large_game.config.num_interceptors,
        large_game.action_info['num_distinct_actions'],
        large_game.config.compute_max_game_length()
    ]
})

print(comparison.to_string(index=False))


      Parameter  Small  Large
      Grid Rows     12     32
      Grid Cols     12     24
        Targets      2      5
       AD Units      1      4
         Drones      5     20
   Interceptors      3     12
   Action Space    208   1271
Max Game Length     13     46


## 4. Playing the Games

Initialize states and explore the game phases.


In [7]:
# Initialize states
small_state = small_game.new_initial_state()
large_state = large_game.new_initial_state()

print("=== SMALL GAME STATE ===")
print(f"Current player: {small_state.current_player()}")
print(f"Phase: {small_state.phase()}")
print(f"Is terminal: {small_state.is_terminal()}")
print(f"Legal actions: {len(small_state.legal_actions())} available")

print("\n=== LARGE GAME STATE ===")
print(f"Current player: {large_state.current_player()}")
print(f"Phase: {large_state.phase()}")
print(f"Is terminal: {large_state.is_terminal()}")
print(f"Legal actions: {len(large_state.legal_actions())} available")

# Show tensor sizes
print("\n=== STATE TENSOR SIZES ===")
print(f"Small game: {len(small_state.information_state_tensor(0))} features")
print(f"Large game: {len(large_state.information_state_tensor(0))} features")


=== SMALL GAME STATE ===
Current player: PlayerId.CHANCE
Phase: Phase.TARGET_POSITIONS
Is terminal: False
Legal actions: 72 available

=== LARGE GAME STATE ===
Current player: PlayerId.CHANCE
Phase: Phase.TARGET_POSITIONS
Is terminal: False
Legal actions: 384 available

=== STATE TENSOR SIZES ===
Small game: 50 features
Large game: 158 features


## Summary

You've now created two different game configurations:

1. **Small Game**: Perfect for rapid prototyping and debugging
   - Faster training cycles
   - Smaller action space
   - Quick to explore

2. **Large Game**: More realistic military scenario
   - Closer to real-world complexity
   - Larger action/state space
   - More strategic depth

**Next Steps:**
- Use these configurations with ESCHER-Torch for training
- Experiment with different parameter combinations
- Save configs to JSON files for reproducibility
- Train policies and compare performance across configurations
